In [257]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv


# First glimpse of the dataset

In [258]:
df = pd.read_csv('../input/heart-disease-prediction-using-logistic-regression/framingham.csv')
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [259]:
print("The dataset is consisted of {} entries and {} features".format(df.shape[0], df.shape[1]))

The dataset is consisted of 4238 entries and 16 features


In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4238 non-null   int64  
 1   age              4238 non-null   int64  
 2   education        4133 non-null   float64
 3   currentSmoker    4238 non-null   int64  
 4   cigsPerDay       4209 non-null   float64
 5   BPMeds           4185 non-null   float64
 6   prevalentStroke  4238 non-null   int64  
 7   prevalentHyp     4238 non-null   int64  
 8   diabetes         4238 non-null   int64  
 9   totChol          4188 non-null   float64
 10  sysBP            4238 non-null   float64
 11  diaBP            4238 non-null   float64
 12  BMI              4219 non-null   float64
 13  heartRate        4237 non-null   float64
 14  glucose          3850 non-null   float64
 15  TenYearCHD       4238 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 529.9 KB


In [261]:
df.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

# Filling missing data

I decided to drop education because it seems not related to the target we want to predict

In [262]:
df = df.drop(['education'],axis=1)

In [263]:
df.cigsPerDay.describe()

count    4209.000000
mean        9.003089
std        11.920094
min         0.000000
25%         0.000000
50%         0.000000
75%        20.000000
max        70.000000
Name: cigsPerDay, dtype: float64

In [264]:
df['cigsPerDay'].fillna(df['cigsPerDay'].median(), inplace=True)

In [265]:
df.BPMeds.value_counts()

0.0    4061
1.0     124
Name: BPMeds, dtype: int64

In [266]:
df['BPMeds'].fillna(df['BPMeds'].value_counts().index[0], inplace=True)

In [267]:
df.totChol.describe()

count    4188.000000
mean      236.721585
std        44.590334
min       107.000000
25%       206.000000
50%       234.000000
75%       263.000000
max       696.000000
Name: totChol, dtype: float64

In [268]:
df['totChol'].fillna(df['totChol'].mean(), inplace=True)

In [269]:
df.BMI.describe()

count    4219.000000
mean       25.802008
std         4.080111
min        15.540000
25%        23.070000
50%        25.400000
75%        28.040000
max        56.800000
Name: BMI, dtype: float64

In [270]:
df['BMI'].fillna(df['BMI'].mean(), inplace=True)

In [271]:
df.heartRate.describe()

count    4237.000000
mean       75.878924
std        12.026596
min        44.000000
25%        68.000000
50%        75.000000
75%        83.000000
max       143.000000
Name: heartRate, dtype: float64

In [272]:
df['heartRate'].fillna(df['heartRate'].value_counts().index[0], inplace=True)

In [273]:
df.glucose.describe()

count    3850.000000
mean       81.966753
std        23.959998
min        40.000000
25%        71.000000
50%        78.000000
75%        87.000000
max       394.000000
Name: glucose, dtype: float64

In [274]:
df['glucose'].fillna(df['glucose'].mean(), inplace=True)

In [275]:
df.isnull().sum()

male               0
age                0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

# Fitting Default Model

I first fit the data into logistic regression model without modifying any of its features

In [276]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from statsmodels.tools import add_constant
import warnings
warnings.filterwarnings('ignore')

X = df.drop(['TenYearCHD'], axis=1)
X = add_constant(X)
y = df['TenYearCHD']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

logReg = LogisticRegression().fit(X_train, y_train)

train_pred = logReg.predict(X_train)
test_pred = logReg.predict(X_test)

print('Train set accuracy score:', accuracy_score(y_train, train_pred))
print('Test set accuracy score:', accuracy_score(y_test, test_pred))

Train set accuracy score: 0.8527375707992448
Test set accuracy score: 0.8462264150943396


# Backward Elimination

I'm using backward elimination to select a certain amount of features and drop the rest of them to see if the model can perform better with new dataset

In [277]:
import statsmodels.api as sm

result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.378589
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 4238
Model:                          Logit   Df Residuals:                     4223
Method:                           MLE   Df Model:                           14
Date:                Wed, 01 Jul 2020   Pseudo R-squ.:                  0.1115
Time:                        07:42:32   Log-Likelihood:                -1604.5
converged:                       True   LL-Null:                       -1805.8
Covariance Type:            nonrobust   LLR p-value:                 3.537e-77
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.1716      0.644    -12.685      0.000      -9.434      -6.909
male                0.5043      0.100      5.024      0.000       0.308       0.701
age                 0.0624      0.006     10.102      0.000       0.050       0.074
currentSmoker       0.0138      0.143      0.096      0.923      -0.266       0.294
cigsPerDay          0.0214      0.006      3.794      0.000       0.010       0.032
BPMeds              0.2423      0.220      1.101      0.271      -0.189       0.674
prevalentStroke     0.9668      0.441      2.191      0.028       0.102       1.832
prevalentHyp        0.2303      0.128      1.792      0.073      -0.022       0.482
diabetes            0.1908      0.294      0.649      0.516      -0.385       0.767
totChol             0.0018      0.001      1.778      0.075      -0.000       0.004
sysBP               0.0142      0.004      4.021      0.000       0.007       0.021
diaBP              -0.0030      0.006     -0.501      0.616      -0.015       0.009
BMI                 0.0038      0.012      0.324      0.746      -0.019       0.027
heartRate          -0.0015      0.004     -0.378      0.705      -0.009       0.006
glucose             0.0067      0.002      3.124      0.002       0.002       0.011
===================================================================================
"""

We need to drop one feature with the highest p-value among those above 0.05 (5%), and continue this process until there is no feature which its p-value is higher than 0.05.

In [278]:
X.drop(['currentSmoker'], axis=1, inplace=True)

result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.378590
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 4238
Model:                          Logit   Df Residuals:                     4224
Method:                           MLE   Df Model:                           13
Date:                Wed, 01 Jul 2020   Pseudo R-squ.:                  0.1115
Time:                        07:42:32   Log-Likelihood:                -1604.5
converged:                       True   LL-Null:                       -1805.8
Covariance Type:            nonrobust   LLR p-value:                 6.246e-78
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.1629      0.638    -12.800      0.000      -9.413      -6.913
male                0.5042      0.100      5.023      0.000       0.307       0.701
age                 0.0623      0.006     10.120      0.000       0.050       0.074
cigsPerDay          0.0218      0.004      5.585      0.000       0.014       0.029
BPMeds              0.2424      0.220      1.101      0.271      -0.189       0.674
prevalentStroke     0.9663      0.441      2.190      0.029       0.102       1.831
prevalentHyp        0.2301      0.128      1.791      0.073      -0.022       0.482
diabetes            0.1904      0.294      0.648      0.517      -0.386       0.767
totChol             0.0018      0.001      1.777      0.076      -0.000       0.004
sysBP               0.0142      0.004      4.022      0.000       0.007       0.021
diaBP              -0.0030      0.006     -0.502      0.615      -0.015       0.009
BMI                 0.0037      0.012      0.315      0.753      -0.019       0.026
heartRate          -0.0015      0.004     -0.375      0.707      -0.009       0.006
glucose             0.0067      0.002      3.124      0.002       0.002       0.011
===================================================================================
"""

In [279]:
X.drop(['BMI'], axis=1, inplace=True)

result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.378601
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 4238
Model:                          Logit   Df Residuals:                     4225
Method:                           MLE   Df Model:                           12
Date:                Wed, 01 Jul 2020   Pseudo R-squ.:                  0.1115
Time:                        07:42:32   Log-Likelihood:                -1604.5
converged:                       True   LL-Null:                       -1805.8
Covariance Type:            nonrobust   LLR p-value:                 1.105e-78
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.0990      0.604    -13.406      0.000      -9.283      -6.915
male                0.5056      0.100      5.042      0.000       0.309       0.702
age                 0.0623      0.006     10.119      0.000       0.050       0.074
cigsPerDay          0.0217      0.004      5.575      0.000       0.014       0.029
BPMeds              0.2439      0.220      1.109      0.268      -0.187       0.675
prevalentStroke     0.9672      0.441      2.194      0.028       0.103       1.831
prevalentHyp        0.2329      0.128      1.817      0.069      -0.018       0.484
diabetes            0.1961      0.293      0.669      0.504      -0.379       0.771
totChol             0.0018      0.001      1.783      0.075      -0.000       0.004
sysBP               0.0142      0.004      4.021      0.000       0.007       0.021
diaBP              -0.0027      0.006     -0.454      0.650      -0.014       0.009
heartRate          -0.0014      0.004     -0.373      0.709      -0.009       0.006
glucose             0.0067      0.002      3.126      0.002       0.003       0.011
===================================================================================
"""

In [280]:
X.drop(['heartRate'], axis=1, inplace=True)

result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.378618
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 4238
Model:                          Logit   Df Residuals:                     4226
Method:                           MLE   Df Model:                           11
Date:                Wed, 01 Jul 2020   Pseudo R-squ.:                  0.1114
Time:                        07:42:32   Log-Likelihood:                -1604.6
converged:                       True   LL-Null:                       -1805.8
Covariance Type:            nonrobust   LLR p-value:                 1.907e-79
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.1868      0.557    -14.707      0.000      -9.278      -7.096
male                0.5105      0.099      5.137      0.000       0.316       0.705
age                 0.0625      0.006     10.171      0.000       0.050       0.075
cigsPerDay          0.0215      0.004      5.575      0.000       0.014       0.029
BPMeds              0.2472      0.220      1.125      0.261      -0.184       0.678
prevalentStroke     0.9720      0.441      2.207      0.027       0.109       1.835
prevalentHyp        0.2304      0.128      1.800      0.072      -0.020       0.481
diabetes            0.1970      0.293      0.672      0.502      -0.378       0.772
totChol             0.0018      0.001      1.763      0.078      -0.000       0.004
sysBP               0.0141      0.004      4.007      0.000       0.007       0.021
diaBP              -0.0028      0.006     -0.473      0.636      -0.014       0.009
glucose             0.0067      0.002      3.108      0.002       0.002       0.011
===================================================================================
"""

In [281]:
X.drop(['diaBP'], axis=1, inplace=True)

result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.378644
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 4238
Model:                          Logit   Df Residuals:                     4227
Method:                           MLE   Df Model:                           10
Date:                Wed, 01 Jul 2020   Pseudo R-squ.:                  0.1114
Time:                        07:42:32   Log-Likelihood:                -1604.7
converged:                       True   LL-Null:                       -1805.8
Covariance Type:            nonrobust   LLR p-value:                 3.264e-80
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.3047      0.498    -16.663      0.000      -9.282      -7.328
male                0.5042      0.098      5.121      0.000       0.311       0.697
age                 0.0631      0.006     10.557      0.000       0.051       0.075
cigsPerDay          0.0216      0.004      5.606      0.000       0.014       0.029
BPMeds              0.2481      0.220      1.130      0.259      -0.182       0.679
prevalentStroke     0.9705      0.440      2.204      0.028       0.107       1.834
prevalentHyp        0.2207      0.126      1.748      0.080      -0.027       0.468
diabetes            0.2004      0.293      0.684      0.494      -0.374       0.775
totChol             0.0018      0.001      1.761      0.078      -0.000       0.004
sysBP               0.0130      0.003      4.857      0.000       0.008       0.018
glucose             0.0067      0.002      3.122      0.002       0.002       0.011
===================================================================================
"""

In [282]:
X.drop(['diabetes'], axis=1, inplace=True)

result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.378698
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 4238
Model:                          Logit   Df Residuals:                     4228
Method:                           MLE   Df Model:                            9
Date:                Wed, 01 Jul 2020   Pseudo R-squ.:                  0.1112
Time:                        07:42:32   Log-Likelihood:                -1604.9
converged:                       True   LL-Null:                       -1805.8
Covariance Type:            nonrobust   LLR p-value:                 5.939e-81
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.3859      0.484    -17.311      0.000      -9.335      -7.436
male                0.5057      0.098      5.138      0.000       0.313       0.699
age                 0.0632      0.006     10.574      0.000       0.051       0.075
cigsPerDay          0.0215      0.004      5.593      0.000       0.014       0.029
BPMeds              0.2516      0.220      1.146      0.252      -0.179       0.682
prevalentStroke     0.9657      0.440      2.194      0.028       0.103       1.828
prevalentHyp        0.2213      0.126      1.752      0.080      -0.026       0.469
totChol             0.0018      0.001      1.765      0.078      -0.000       0.004
sysBP               0.0131      0.003      4.866      0.000       0.008       0.018
glucose             0.0076      0.002      4.662      0.000       0.004       0.011
===================================================================================
"""

In [283]:
X.drop(['BPMeds'], axis=1, inplace=True)

result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.378850
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 4238
Model:                          Logit   Df Residuals:                     4229
Method:                           MLE   Df Model:                            8
Date:                Wed, 01 Jul 2020   Pseudo R-squ.:                  0.1109
Time:                        07:42:32   Log-Likelihood:                -1605.6
converged:                       True   LL-Null:                       -1805.8
Covariance Type:            nonrobust   LLR p-value:                 1.527e-81
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.4350      0.483    -17.478      0.000      -9.381      -7.489
male                0.5011      0.098      5.097      0.000       0.308       0.694
age                 0.0633      0.006     10.581      0.000       0.052       0.075
cigsPerDay          0.0215      0.004      5.583      0.000       0.014       0.029
prevalentStroke     1.0185      0.438      2.325      0.020       0.160       1.877
prevalentHyp        0.2339      0.126      1.862      0.063      -0.012       0.480
totChol             0.0018      0.001      1.793      0.073      -0.000       0.004
sysBP               0.0134      0.003      5.028      0.000       0.008       0.019
glucose             0.0076      0.002      4.669      0.000       0.004       0.011
===================================================================================
"""

In [284]:
X.drop(['prevalentHyp'], axis=1, inplace=True)

result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.379257
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 4238
Model:                          Logit   Df Residuals:                     4230
Method:                           MLE   Df Model:                            7
Date:                Wed, 01 Jul 2020   Pseudo R-squ.:                  0.1099
Time:                        07:42:33   Log-Likelihood:                -1607.3
converged:                       True   LL-Null:                       -1805.8
Covariance Type:            nonrobust   LLR p-value:                 1.065e-81
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.8181      0.438    -20.138      0.000      -9.676      -7.960
male                0.5081      0.098      5.174      0.000       0.316       0.701
age                 0.0636      0.006     10.652      0.000       0.052       0.075
cigsPerDay          0.0214      0.004      5.558      0.000       0.014       0.029
prevalentStroke     1.0642      0.437      2.436      0.015       0.208       1.920
totChol             0.0019      0.001      1.837      0.066      -0.000       0.004
sysBP               0.0167      0.002      8.311      0.000       0.013       0.021
glucose             0.0076      0.002      4.622      0.000       0.004       0.011
===================================================================================
"""

In [285]:
X.drop(['totChol'], axis=1, inplace=True)

result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.379650
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 4238
Model:                          Logit   Df Residuals:                     4231
Method:                           MLE   Df Model:                            6
Date:                Wed, 01 Jul 2020   Pseudo R-squ.:                  0.1090
Time:                        07:42:33   Log-Likelihood:                -1609.0
converged:                       True   LL-Null:                       -1805.8
Covariance Type:            nonrobust   LLR p-value:                 6.519e-82
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.4651      0.390    -21.730      0.000      -9.229      -7.702
male                0.4844      0.097      4.983      0.000       0.294       0.675
age                 0.0647      0.006     10.927      0.000       0.053       0.076
cigsPerDay          0.0216      0.004      5.623      0.000       0.014       0.029
prevalentStroke     1.0465      0.436      2.399      0.016       0.192       1.901
sysBP               0.0171      0.002      8.529      0.000       0.013       0.021
glucose             0.0076      0.002      4.648      0.000       0.004       0.011
===================================================================================
"""

In [286]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

logReg = LogisticRegression().fit(X_train, y_train)
train_pred = logReg.predict(X_train)
test_pred = logReg.predict(X_test)

print('New train set accuracy:', accuracy_score(y_train, train_pred))
print('New test set accuracy:', accuracy_score(y_test, test_pred))

New train set accuracy: 0.8580868470736313
New test set accuracy: 0.8424528301886792


In [287]:
confusion_matrix(y_test, test_pred)

array([[879,   8],
       [159,  14]])

After performing backward elimination, there is no significant improvement in prediction accuracy, either training and test set. The problem could be the way I performed backward elimination with the feature chosen. 